In [1]:
import numpy as np
import pandas as pd
import re
#visual
import matplotlib.pyplot as plt
import cv2 as cv
import scipy as sy
from PIL import Image

#file
import shutil
import os
import pathlib
import glob

#models
import tensorflow_hub as hub
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

from skimage.feature import greycomatrix,greycoprops
from skimage import io


/home/tama/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


### mask

In [2]:
SIZE = 224

train_image_mask= []
train_label_mask = []

for directory_path in glob.glob("../data/img_cyt_train/*"):
  label = directory_path.split("\\")[-1]
  #print()
  for img_path in glob.glob(os.path.join(directory_path,"*.bmp")):
    #print(img_path)
    img = cv.imread(img_path, cv.IMREAD_COLOR)
    img = cv.resize(img,(SIZE,SIZE))
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    train_image_mask.append(img)
    train_label_mask.append(label)

train_images_mask = np.array(train_image_mask)
train_labels_mask = np.array(train_label_mask)

In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels_mask)
train_label_endc_mask = le.transform(train_labels_mask)



x_train_mask, y_train_mask = train_images_mask, train_label_endc_mask

x_train_mask = x_train_mask/225

np.savetxt('mask/mobilev2/dense_label_train_mask.txt', y_train_mask)

In [4]:
IMG_SHAPE = (224,224) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

2022-11-02 00:13:05.800693: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
feature_extraction_mobile = base_model.predict(x_train_mask[:500], batch_size=10)
print(feature_extraction_mobile.shape)

2022-11-02 00:13:07.596646: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 301056000 exceeds 10% of free system memory.
2022-11-02 00:13:09.178900: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 48168960 exceeds 10% of free system memory.
2022-11-02 00:13:09.194491: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 49032960 exceeds 10% of free system memory.
2022-11-02 00:13:09.226243: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18063360 exceeds 10% of free system memory.
2022-11-02 00:13:09.233044: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18063360 exceeds 10% of free system memory.


50/50 [==============================] - 13s 242ms/step
(500, 7, 7, 1280)


In [6]:
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

(500, 62720)

In [7]:
np.savetxt("mask/mobilev2/pre_train1.txt",feature_extraction_mobile)

In [8]:
feature_extraction_mobile = base_model.predict(x_train_mask[500:1000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 247ms/step


(500, 62720)

In [9]:
np.savetxt("mask/mobilev2/pre_train2.txt",feature_extraction_mobile)

In [10]:
feature_extraction_mobile = base_model.predict(x_train_mask[1000:1500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 247ms/step


(500, 62720)

In [11]:
np.savetxt("mask/mobilev2/pre_train3.txt",feature_extraction_mobile)

In [12]:
feature_extraction_mobile = base_model.predict(x_train_mask[1500:2000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 244ms/step


(500, 62720)

In [13]:
np.savetxt("mask/mobilev2/pre_train4.txt",feature_extraction_mobile)

In [14]:
feature_extraction_mobile = base_model.predict(x_train_mask[2000:2500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 12s 247ms/step


(500, 62720)

In [15]:
np.savetxt("mask/mobilev2/pre_train5.txt",feature_extraction_mobile)

In [16]:
feature_extraction_mobile = base_model.predict(x_train_mask[2500:3000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 11s 222ms/step


(500, 62720)

In [17]:
np.savetxt("mask/mobilev2/pre_train6.txt",feature_extraction_mobile)

In [18]:
feature_extraction_mobile = base_model.predict(x_train_mask[3000:], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

105/105 [==============================] - 24s 213ms/step


(1047, 62720)

In [19]:
np.savetxt("mask/mobilev2/pre_train7.txt",feature_extraction_mobile)

In [ ]:
dt_DenseNet1 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train1.txt')
dt_DenseNet2 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train2.txt')
dt_DenseNet3 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train3.txt')
dt_DenseNet4 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train4.txt')
dt_DenseNet5 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train5.txt')
dt_DenseNet6 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train6.txt')
dt_DenseNet7 = np.loadtxt('mask/DenseNet121/DenseNet_pre_train7.txt')

DenseNet_full = [dt_DenseNet1,dt_DenseNet2,dt_DenseNet3,dt_DenseNet4,dt_DenseNet5,dt_DenseNet6,dt_DenseNet7]

for i in DenseNet_full:print(i.shape)

In [ ]:
train_concat = np.concatenate((dt_DenseNet1, dt_DenseNet2, dt_DenseNet3, dt_DenseNet4, dt_DenseNet5, dt_DenseNet6), axis=0)
np.savetxt("mask/DenseNet121/densenet_full_data.txt",train_concat)

### no mask

In [3]:
SIZE = 224

train_image= []
train_label = []

for directory_path in glob.glob("../data/train/*"):
  label = directory_path.split("\\")[-1]
  #print()
  for img_path in glob.glob(os.path.join(directory_path,"*.bmp")):
    #print(img_path)
    img = cv.imread(img_path, cv.IMREAD_COLOR)
    img = cv.resize(img,(SIZE,SIZE))
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    train_image.append(img)
    train_label.append(label)

train_images_mask = np.array(train_image)
train_labels_mask = np.array(train_label)

In [5]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels_mask)
train_label_endc_mask = le.transform(train_labels_mask)



x_train_mask, y_train_mask = train_images_mask, train_label_endc_mask

x_train_mask = x_train_mask/225

np.savetxt('nomask/mobilev2/dense_label_train.txt', y_train_mask)

In [6]:
IMG_SHAPE = (224,224) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

2023-01-17 00:22:39.197956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
feature_extraction_mobile = base_model.predict(x_train_mask[:500], batch_size=10)
print(feature_extraction_mobile.shape)

feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 13s 262ms/step
(500, 7, 7, 1280)


(500, 62720)

In [11]:
np.savetxt("nomask/mobilev2/pre_train1.txt",feature_extraction_mobile)

In [8]:
feature_extraction_mobile = base_model.predict(x_train_mask[500:1000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 13s 254ms/step


(500, 62720)

In [9]:
np.savetxt("nomask/mobilev2/pre_train2.txt",feature_extraction_mobile)

In [12]:
feature_extraction_mobile = base_model.predict(x_train_mask[1000:1500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 16s 313ms/step


(500, 62720)

In [13]:
np.savetxt("nomask/mobilev2/pre_train3.txt",feature_extraction_mobile)

In [14]:
feature_extraction_mobile = base_model.predict(x_train_mask[1500:2000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 16s 312ms/step


(500, 62720)

In [15]:
np.savetxt("nomask/mobilev2/pre_train4.txt",feature_extraction_mobile)

In [17]:
feature_extraction_mobile = base_model.predict(x_train_mask[2000:2500], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 13s 260ms/step


(500, 62720)

In [18]:
np.savetxt("nomask/mobilev2/pre_train5.txt",feature_extraction_mobile)

In [19]:
feature_extraction_mobile = base_model.predict(x_train_mask[2500:3000], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

50/50 [==============================] - 13s 254ms/step


(500, 62720)

In [20]:
np.savetxt("nomask/mobilev2/pre_train6.txt",feature_extraction_mobile)

In [21]:
feature_extraction_mobile = base_model.predict(x_train_mask[3000:], batch_size=10)
feature_extraction_mobile = feature_extraction_mobile.reshape(feature_extraction_mobile.shape[0],-1)
feature_extraction_mobile.shape

4/4 [==============================] - 2s 194ms/step


(34, 62720)

In [22]:
np.savetxt("nomask/mobilev2/pre_train7.txt",feature_extraction_mobile)

In [23]:
dt_DenseNet1 = np.loadtxt('nomask/mobilev2/pre_train1.txt')
dt_DenseNet2 = np.loadtxt('nomask/mobilev2/pre_train2.txt')
dt_DenseNet3 = np.loadtxt('nomask/mobilev2/pre_train3.txt')
dt_DenseNet4 = np.loadtxt('nomask/mobilev2/pre_train4.txt')
dt_DenseNet5 = np.loadtxt('nomask/mobilev2/pre_train5.txt')
dt_DenseNet6 = np.loadtxt('nomask/mobilev2/pre_train6.txt')
dt_DenseNet7 = np.loadtxt('nomask/mobilev2/pre_train7.txt')

DenseNet_full = [dt_DenseNet1,dt_DenseNet2,dt_DenseNet3,dt_DenseNet4,dt_DenseNet5,dt_DenseNet6,dt_DenseNet7]

for i in DenseNet_full:print(i.shape)

(500, 62720)
(500, 62720)
(500, 62720)
(500, 62720)
(500, 62720)
(500, 62720)
(34, 62720)


In [24]:
train_concat = np.concatenate((dt_DenseNet1, dt_DenseNet2, dt_DenseNet3, dt_DenseNet4, dt_DenseNet5, dt_DenseNet6), axis=0)
np.savetxt("nomask/mobilev2/mobilev2_full_data.txt",train_concat)

: 